# NasNet Dogs v Cats

In [22]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [23]:
from fastai.conv_learner import *
PATH = "data/dogscats/"
sz=224; bs=48

In [24]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2

In [25]:
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [29]:
learn = ConvLearner.pretrained(nasnet(True), data, precompute=True, xtra_fc=[], ps=0.5)

KeyError: 'unexpected key "last_linear.weight" in state_dict'

In [ ]:
%debug

> /home/bmn/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py(522)load_state_dict()
    520             elif strict:
    521                 raise KeyError('unexpected key "{}" in state_dict'
--> 522                                .format(name))
    523         if strict:
    524             missing = set(own_state.keys()) - set(state_dict.keys())

ipdb> model
*** NameError: name 'model' is not defined
ipdb> up
> /home/bmn/src/ai.learn/fast.ai/fastai/courses/dl1/fastai/models/nasnet.py(604)nasnetalarge()
    602         # both 'imagenet'&'imagenet+background' are loaded from same parameters
    603         model = NASNetALarge(num_classes=1001)
--> 604         model.load_state_dict(model_zoo.load_url(settings['url']))
    605 
    606         if pretrained == 'imagenet':

ipdb> model
NASNetALarge(
  (conv0): Sequential(
    (conv): Conv2d (3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
  )

ipdb> dir(model)
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_buffers', '_apply', '_backend', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_modules', '_parameters', '_slow_forward', '_tracing_name', 'add_module', 'apply', 'cell_0', 'cell_1', 'cell_10', 'cell_11', 'cell_12', 'cell_13', 'cell_14', 'cell_15', 'cell_16', 'cell_17', 'cell_2', 'cell_3', 'cell_4', 'cell_5', 'cell_6', 'cell_7', 'cell_8', 'cell_9', 'cell_stem_0', 'cell_stem_1', 'children', 'classifier', 'conv0', 'cpu', 'cuda', 'double', 'dropout', 'dump_patches', 'eval', 'features', 'float', 'forward', 'half', 'linear', 'load_state_dict', 'modules', 'named_ch

ipdb> model.state_dict()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [1]:
%time learn.fit(1e-2, 2)

In [19]:
learn.precompute=False
learn.bn_freeze=True

In [2]:
%time learn.fit(1e-2, 1, cycle_len=1)

In [21]:
learn.save('nas_pre')

In [28]:
def freeze_to(m, n):
    c=children(m[0])
    for l in c:     set_trainable(l, False)
    for l in c[n:]: set_trainable(l, True)

In [29]:
freeze_to(learn.model, 17)

In [3]:
learn.fit([1e-5,1e-4,1e-2], 1, cycle_len=1)

In [9]:
learn.save('nas')